In [ ]:
#| label: afiFig8jn

# Prepare Python environment

import scipy.io as sio
import numpy as np
from pathlib import Path

from contextlib import contextmanager
import sys, os
from pathlib import Path

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

import os
from pathlib import Path

def find_myst_yml_directories(start_dir=None):
    """
    Recursively search for directories containing myst.yml file.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        list: List of full paths to directories containing myst.yml
    """
    if start_dir is None:
        start_dir = Path.cwd()
    else:
        start_dir = Path(start_dir)
    
    myst_dirs = []
    
    def _search_directory(current_dir):
        # Check if myst.yml exists in current directory
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            myst_dirs.append(str(current_dir.resolve()))
            # Don't search subdirectories if we found myst.yml here
            return
        
        # Recursively search all subdirectories
        for item in current_dir.iterdir():
            if item.is_dir():
                try:
                    _search_directory(item)
                except (PermissionError, OSError):
                    # Skip directories we can't access
                    continue
    
    _search_directory(start_dir)
    return myst_dirs

def find_myst_yml_directories_upwards(start_dir=None):
    """
    Search for myst.yml in current directory, if not found go to parent and repeat.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        str or None: Full path of directory containing myst.yml, or None if not found
    """
    if start_dir is None:
        current_dir = Path.cwd()
    else:
        current_dir = Path(start_dir)
    
    # Keep going up until we reach the filesystem root
    while current_dir != current_dir.parent:  # Stop at root
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            return str(current_dir.resolve())
        
        # Move to parent directory
        current_dir = current_dir.parent
    
    return None

with suppress_stdout():
    repo_path = Path(find_myst_yml_directories_upwards())
    print(repo_path)
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
data_dir = dataset_path / "04-B1-02-AFI" / "04-B1-02-AFI"  

data_file = "fig7.mat"

#Load either archived or generated plot variables
mat_contents = sio.loadmat(data_dir / data_file)

B1map_raw = mat_contents["B1map_raw"]
AFIData1 = mat_contents["AFIData1"]
AFIData2 = mat_contents["AFIData2"]
mask = mat_contents["mask"]
xAxis = mat_contents["xAxis"][0]
yAxis = mat_contents["yAxis"][0]

# Masking B1 map
B1map_raw = np.asarray(B1map_raw)
mask = np.asarray(mask)
B1map_raw_masked = B1map_raw*mask
B1map_raw_masked[np.isnan(B1map_raw_masked)] = 0

data_file = "fig8.mat"


#Load either archived or generated plot variables
mat_contents = sio.loadmat(data_dir / data_file)

B1map_filtered = mat_contents["B1map_filtered"]

## Plot
# PYTHON CODE
# Module imports

import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}
# PYTHON CODE
from plotly import tools

# Masking B1 map
B1map_filtered = np.asarray(B1map_filtered)
mask = np.asarray(mask)
B1map_filtered_masked = B1map_filtered*mask
B1map_filtered_masked[np.isnan(B1map_filtered_masked)] = 0

trace1 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=B1map_raw_masked,
                   zmin=0.7,
                   zmax=1.3,
                   colorscale='RdBu',
                   showscale = False,
                   visible=True,
                   name = 'B1 values')
trace2 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=B1map_filtered_masked,
                   zmin=0.7,
                   zmax=1.3,
                   colorscale='RdBu',
                   xaxis='x2',
                   yaxis='y2',
                   visible=True,
                   name = 'B1 values')

data=[trace1, trace2]

layout = dict(
    width=560,
    height=310,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.04,
            y=1.15,
            showarrow=False,
            text='Raw B<sub>1</sub> map',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.60,
            y=1.15,
            showarrow=False,
            text='Filtered B<sub>1</sub> map',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.12,
            y=1.15,
            showarrow=False,
            text='B<sub>1</sub>',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 0.58]),
    yaxis = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1]),
    xaxis2 = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.40, 0.98]),
    yaxis2 = dict(range = [0,127], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 1], anchor='x2'),
    showlegend = False,
    autosize = False,
)


fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-heatmap', config = config)